# MLP

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('data/', one_hot = True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [3]:
#Network traits
hidden1 = 256
hidden2 = 128
hidden3 = 64
n_inputs = 784
n_classes = 10

#input and output
x = tf.placeholder("float", [None, n_inputs])
y = tf.placeholder("float", [None, n_classes])

#Network paremeter
stddev = 0.1
weights = {
    'h1' : tf.Variable(tf.random_normal([n_inputs, hidden1], stddev = stddev)),
    'h2' : tf.Variable(tf.random_normal([hidden1, hidden2], stddev = stddev)),
    'h3' : tf.Variable(tf.random_normal([hidden2, hidden3], stddev = stddev)),
    'out' : tf.Variable(tf.random_normal([hidden3, n_classes], stddev = stddev))
}
biases = {
    'b1' : tf.Variable(tf.random_normal([hidden1], stddev=stddev)),
    'b2' : tf.Variable(tf.random_normal([hidden2], stddev=stddev)),
    'b3' : tf.Variable(tf.random_normal([hidden3], stddev=stddev)),
    'b4' : tf.Variable(tf.random_normal([n_classes], stddev=stddev))
}
print 'network ready'

network ready


# Define Graph

In [5]:
#Model
def mlp(_X, _weights, _biases):
    layer1 = tf.nn.sigmoid(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1']))
    layer2 = tf.nn.sigmoid(tf.add(tf.matmul(layer1, _weights['h2']), _biases['b2']))
    layer3 = tf.nn.sigmoid(tf.add(tf.matmul(layer2, _weights['h3']), _biases['b3']))
    return (tf.matmul(layer3, _weights['out']) + _biases['b4'])

#prediction
pred = mlp(x, weights, biases)

#loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
corr = tf.equal(tf.arg_max(y,1), tf.arg_max(pred,1))
accr = tf.reduce_mean(tf.cast(corr, 'float'))

init = tf.global_variables_initializer()
print ("Function ready")

Function ready


# Run

In [6]:
training_epochs = 10
batch_size      = 100
display_step    = 2
# SESSION
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# MINI-BATCH LEARNING
for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(mnist.train.num_examples/batch_size)
    for i in range(num_batch): 
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys})
        feeds = {x: batch_xs, y: batch_ys}
        avg_cost += sess.run(cost, feed_dict=feeds)/num_batch
    # DISPLAY
    if epoch % display_step == 0:
        feeds_train = {x: batch_xs, y: batch_ys}
        feeds_test = {x: mnist.test.images, y: mnist.test.labels}
        train_acc = sess.run(accr, feed_dict=feeds_train)
        test_acc = sess.run(accr, feed_dict=feeds_test)
        print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f" 
               % (epoch, training_epochs, avg_cost, train_acc, test_acc))
print ("DONE")

Epoch: 000/010 cost: 0.766358453 train_acc: 0.940 test_acc: 0.917
Epoch: 002/010 cost: 0.151252287 train_acc: 0.950 test_acc: 0.961
Epoch: 004/010 cost: 0.084785870 train_acc: 0.990 test_acc: 0.970
Epoch: 006/010 cost: 0.052952738 train_acc: 0.990 test_acc: 0.972
Epoch: 008/010 cost: 0.032347464 train_acc: 0.990 test_acc: 0.976
DONE
